<a href="https://colab.research.google.com/github/NousForFreedom/taxi_call_service_pubsub/blob/main/taxi_payment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install solace-pubsubplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.0 MB/s eta 0:00:00


In [ ]:
import os
import platform
import time
import calendar
import math

# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
            ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.request_reply_message_publisher import RequestReplyMessagePublisher

# Import Solace Python  API modules
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler
from solace.messaging.config.solace_properties.message_properties import APPLICATION_MESSAGE_ID
from solace.messaging.resources.topic import Topic
from solace.messaging.receiver.inbound_message import InboundMessage



In [ ]:
# Broker Config
broker_props = {
  "solace.messaging.transport.host": "tcp://mr-connection-hcnx0u899eb.messaging.solace.cloud:55555",
  "solace.messaging.service.vpn-name": "aiot",
  "solace.messaging.authentication.scheme.basic.username": "Hanbinheo",
  "solace.messaging.authentication.scheme.basic.password": "ajou",
}

In [ ]:
from solace.messaging.receiver.request_reply_message_receiver import RequestMessageHandler

TOPIC_PREFIX = "solace/taxi/python"

# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a request-reply message receiver for receiving PaymentRequest from the platform
request_reply_receiver = messaging_service.request_reply() \
                                            .create_request_reply_message_receiver_builder() \
                                            .build(TopicSubscription.of(TOPIC_PREFIX + '/direct/payment/>'))

request_reply_receiver.start()

# Handler for processing PaymentRequest from the platform
class PaymentRequestHandler(RequestMessageHandler):
    def on_message(self, request, replier):
        request_payload = request.get_payload_as_string()
        print(f'Received PaymentRequest: {request_payload}')

        try:
            details = {part.split(': ')[0]: part.split(': ')[1] for part in request_payload.split(', ')}
            timestamp = details.get("timestamp")
            cost = details.get("cost")
            account = details.get("account")

            #print(f'PaymentRequest details: Timestamp {timestamp}, Cost {cost}, Account {account}')

        except Exception as e:
            print(f'Failed to parse payload: {request_payload} with error: {e}')

# Register the request handler
request_handler = PaymentRequestHandler()
request_reply_receiver.receive_async(request_handler)

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print('\nTerminating Request-Reply Receiver')
    request_reply_receiver.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()

Messaging Service connected? True
Received PaymentRequest: user_id: USER12345, ride_id: RIDE12345
Received PaymentRequest: user_id: USER12345, ride_id: RIDE12345
Received PaymentRequest: user_id: USER12345, ride_id: RIDE12345
Received PaymentRequest: user_id: USER12345, ride_id: RIDE12345


In [ ]:
# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message publisher for sending PaymentRequest to the user
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
print(f'Direct Publisher ready? {direct_publisher.is_ready()}')

# Define the topic to send PaymentRequest message to the user
user_payment_topic = Topic.of(TOPIC_PREFIX + f'/direct/payment/request/{user_id}')

# Build the message payload
timestamp = str(calendar.timegm(time.gmtime()))
cost = "100"  # Example cost
account = "1234567890"  # Example account

payload = f"timestamp: {timestamp}, cost: {cost}, account: {account}"

try:
    outbound_msg = messaging_service.message_builder() \
        .with_application_message_id(f'PaymentRequest_{USER_ID}_{timestamp}') \
        .build(payload)

    direct_publisher.publish(destination=user_payment_topic, message=outbound_msg)
    print(f'Sent PaymentRequest message to topic: {user_payment_topic}')
except Exception as e:
    print(f'Failed to send PaymentRequest message: {e}')

finally:
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()